# 셀프 쿼리(Self-querying)

`SelfQueryRetriever` 는 자체적으로 질문을 생성하고 해결할 수 있는 기능을 갖춘 검색 도구입니다. 이는 사용자가 제공한 자연어 질의를 바탕으로, query-constructing LLM chain을 사용해 구조화된 질의를 만듭니다. 그 후, 이 구조화된 질의를 기본 벡터 데이터 저장소(VectorStore)에 적용하여 검색을 수행합니다.

이 과정을 통해, `SelfQueryRetriever` 는 단순히 사용자의 입력 질의를 저장된 문서의 내용과 의미적으로 비교하는 것을 넘어서, 사용자의 질의에서 문서의 메타데이터에 대한 필터를 추출하고, 이 필터를 실행하여 관련된 문서를 찾을 수 있습니다. 이를 통해, 사용자의 질의에 대한 더 정확하고 관련성 높은 결과를 제공할 수 있게 됩니다.

[참고]

- LangChain 이 지원하는 셀프 쿼리 번역기(Self-query Retriever) 목록은 [여기](https://python.langchain.com/docs/integrations/retrievers/self_query) 에서 확인해 주시기 바랍니다.


## 시작하기

데모를 위해 `Chroma` vector store를 사용할 예정입니다. 이번 튜토리얼에서는 영화 요약이 포함된 작은 데모 문서 세트를 만들었습니다.

**참고:** `SelfQueryRetriever` 를 사용하려면 `lark` 패키지를 설치해야 합니다.


In [ ]:
# 설치
%pip install -qU lark chromadb

영화 설명과 메타데이터를 기반으로 유사도 검색이 가능한 벡터 저장소가 구축합니다.

- `Document` 클래스를 사용하여 영화에 대한 간략한 설명과 메타데이터를 포함하는 문서 객체 리스트인 `docs`를 생성합니다.
- `OpenAIEmbeddings`를 사용하여 문서 임베딩을 생성합니다.
- `Chroma.from_documents` 메서드를 사용하여 `docs`와 `OpenAIEmbeddings`로부터 Chroma 벡터 저장소인 `vectorstore`를 생성합니다.


In [1]:
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings


docs = [
    Document(
        page_content="A bunch of scientists bring back dinosaurs and mayhem breaks loose",
        metadata={"year": 1993, "rating": 7.7, "genre": "science fiction"},
    ),
    Document(
        page_content="Leo DiCaprio gets lost in a dream within a dream within a dream within a ...",
        metadata={"year": 2010, "director": "Christopher Nolan", "rating": 8.2},
    ),
    Document(
        page_content="A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea",
        metadata={"year": 2006, "director": "Satoshi Kon", "rating": 8.6},
    ),
    Document(
        page_content="A bunch of normal-sized women are supremely wholesome and some men pine after them",
        metadata={"year": 2019, "director": "Greta Gerwig", "rating": 8.3},
    ),
    Document(
        page_content="Toys come alive and have a blast doing so",
        metadata={"year": 1995, "genre": "animated"},
    ),
    Document(
        page_content="Three men walk into the Zone, three men walk out of the Zone",
        metadata={
            "year": 1979,
            "director": "Andrei Tarkovsky",
            "genre": "thriller",
            "rating": 9.9,
        },
    ),
]
vectorstore = Chroma.from_documents(docs, OpenAIEmbeddings())

## SelfQueryRetriever 생성하기

이제 retriever를 인스턴스화할 수 있습니다. 이를 위해서는 문서가 지원하는 **메타데이터 필드** 와 문서 내용에 대한 **간단한 설명을 미리 제공** 해야 합니다.


`AttributeInfo` 클래스를 사용하여 영화 메타데이터 필드에 대한 정보를 정의합니다.

- 장르(`genre`): 문자열 타입, 영화의 장르를 나타내며 ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated'] 중 하나의 값을 가집니다.
- 연도(`year`): 정수 타입, 영화가 개봉된 연도를 나타냅니다.
- 감독(`director`): 문자열 타입, 영화 감독의 이름을 나타냅니다.
- 평점(`rating`): 실수 타입, 1-10 범위의 영화 평점을 나타냅니다.


In [14]:
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

metadata_field_info = [
    AttributeInfo(
        name="genre",
        description="The genre of the movie. One of ['science fiction', 'comedy', 'drama', 'thriller', 'romance', 'action', 'animated']",
        type="string",
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released",
        type="integer",
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie director",
        type="string",
    ),
    AttributeInfo(
        name="rating", description="A 1-10 rating for the movie", type="float"
    ),
]

`document_content_description` 변수에 영화에 대한 간략한 요약 설명을 할당합니다.


In [16]:
# 문서의 내용에 대한 간략한 설명
document_content_description = "Brief summary of a movie"

`SelfQueryRetriever.from_llm()` 메서드를 사용하여 `retriever` 객체를 생성합니다.

- `llm`: 언어 모델
- `vectorstore`: 벡터 저장소
- `document_content_description`: 문서 내용 설명
- `metadata_field_info`: 메타데이터 필드 정보


In [17]:
# LLM 정의
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)

# SelfQueryRetriever 생성
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectorstore,
    document_content_description,
    metadata_field_info,
)

## 테스트

이제 우리가 만든 retriever를 실제로 사용해 볼 수 있습니다!


`retriever` 객체의 `invoke` 메서드를 호출하여 필터링된 검색을 수행합니다.

- 검색 쿼리로 "I want to watch a movie rated higher than 8.5" 를 전달하여 평점이 8.5 이상인 영화를 검색하도록 지정합니다.


In [18]:
# 8.5 이상의 평점을 받은 영화를 보고 싶다는 필터만 지정합니다.
retriever.invoke("I want to watch a movie rated higher than 8.5")

[Document(page_content='Three men walk into the Zone, three men walk out of the Zone', metadata={'director': 'Andrei Tarkovsky', 'genre': 'thriller', 'rating': 9.9, 'year': 1979}),
 Document(page_content='A psychologist / detective gets lost in a series of dreams within dreams within dreams and Inception reused the idea', metadata={'director': 'Satoshi Kon', 'rating': 8.6, 'year': 2006})]

질의 내용은 "Has Greta Gerwig directed any movies about women"으로, Greta Gerwig가 여성에 관한 영화를 연출했는지 묻는 질문입니다.


In [19]:
# Greta Gerwig가 여성에 관한 영화를 연출한 적이 있는지 질의합니다.
retriever.invoke("Has Greta Gerwig directed any movies about women")

[Document(page_content='A bunch of normal-sized women are supremely wholesome and some men pine after them', metadata={'director': 'Greta Gerwig', 'rating': 8.3, 'year': 2019})]

이번 쿼리에서는 **복합 필터(composite filter)** 를 사용하여 검색 조건을 지정했음을 알 수 있습니다.

- 복합 필터: rating above 8.5, science fiction


In [20]:
# 8.5 이상의 평점을 가진 SF 영화를 검색하는 복합 필터를 지정합니다.
retriever.invoke("What's a highly rated (above 8.5) science fiction film?")

[]

이 질의 역시 복합 필터를 사용하여 검색 결과를 필터링합니다.

- 복합 필터: 1990 ~ 2005년, 장난관 관련 영화, 에니메이션 영화 선호


In [ ]:
# 에러가 발생하는 쿼리
retriever.invoke(
    # 1990년 이후 2005년 이전에 제작된 장난감에 관한 영화를 검색하며, 애니메이션 영화를 선호한다는 쿼리와 복합 필터를 지정합니다.
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

하지만, retriever.invoke()를 호출할 때마다 에러가 발생합니다.

이 문제를 해결하기 위한 방법은 노트북의 제일 하단에 나옵니다.


## K 계수 필터링

`k`는 가져올 문서의 수를 의미합니다.

`SelfQueryRetriever`를 사용하여 `k`를 지정할 수도 있습니다. 이는 생성자에 `enable_limit=True`를 전달하여 수행할 수 있습니다.


`SelfQueryRetriever` 클래스를 사용하여 `retriever` 객체를 생성합니다.

- `document_content_description`: 문서 내용에 대한 설명
- `metadata_field_info`: 메타데이터 필드 정보
- `enable_limit`: 검색 결과 제한 여부


(방법1) 아래의 경우는 `search_kwargs={"k": 2}` 을 명시적으로 지정하여 2개의 검색 결과를 반환하도록 지정합니다.


In [29]:
retriever = SelfQueryRetriever.from_llm(
    llm,  # 언어 모델(Language Model)을 지정합니다.
    vectorstore,  # 벡터 저장소(Vector Store)를 지정합니다.
    document_content_description,  # 문서 내용에 대한 설명을 지정합니다.
    metadata_field_info,  # 메타데이터 필드 정보를 지정합니다.
    enable_limit=True,  # 검색 결과 제한 기능을 활성화합니다.
    search_kwargs={"k": 2},  # k 의 값을 2로 지정하여 검색 결과를 2개로 제한합니다.
)

# 공룡에 관한 두 영화가 무엇인지 질의합니다.
retriever.invoke("What are movies about dinosaurs")

[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995})]

하지만 코드로 명시적으로 `search_kwargs`를 지정하지 않고 query 에서 `two, three` 등의 숫자를 사용하여 검색 결과를 제한할 수 있습니다.


In [30]:
retriever = SelfQueryRetriever.from_llm(
    llm,  # 언어 모델(Language Model)을 지정합니다.
    vectorstore,  # 벡터 저장소(Vector Store)를 지정합니다.
    document_content_description,  # 문서 내용에 대한 설명을 지정합니다.
    metadata_field_info,  # 메타데이터 필드 정보를 지정합니다.
    enable_limit=True,  # 검색 결과 제한 기능을 활성화합니다.
)

# 공룡에 관한 두 영화가 무엇인지 질의합니다.(two)
retriever.invoke("What are two movies about dinosaurs")

[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993}),
 Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995})]

하나만 반환하도록 질의 내용을 조금 수정해 보겠습니다.


In [31]:
# 공룡에 관한 두 영화가 무엇인지 질의합니다.(one)
retriever.invoke("What are one movies about dinosaurs")

[Document(page_content='A bunch of scientists bring back dinosaurs and mayhem breaks loose', metadata={'genre': 'science fiction', 'rating': 7.7, 'year': 1993})]

## LCEL을 사용하여 Chain 구성하기

내부에서 어떤 일이 일어나는지 확인하고 더 많은 사용자 정의 제어를 하기 위해, 우리는 retriever를 처음부터 재구성할 수 있습니다.

먼저, query-construction chain을 생성해야 합니다.

이 chain은 사용자 query를 받아 사용자가 지정한 필터를 캡처하는 `StructuredQuery` 객체를 생성합니다.


### ① 구조화된 쿼리 생성기(query_constructor) 생성


`get_query_constructor_prompt` 함수를 사용하여 쿼리 생성기 프롬프트를 가져옵니다.

- 이 함수는 `document_content_description`과 `metadata_field_info`를 인자로 받습니다.

`StructuredQueryOutputParser.from_components()` 메서드를 사용하여 구조화된 쿼리 출력 파서를 초기화합니다.

- 쿼리 생성기 프롬프트(`prompt`)와 언어 모델(`llm`), 그리고 출력 파서(`output_parser`)를 파이프라인으로 연결하여 `query_constructor`를 생성합니다.
- 이 파이프라인은 프롬프트에 따라 쿼리를 생성하고, 언어 모델을 통해 처리한 후, 출력 파서를 사용하여 구조화된 형식으로 변환합니다.


In [32]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

# 문서 내용 설명과 메타데이터 필드 정보를 사용하여 쿼리 생성기 프롬프트를 가져옵니다.
prompt = get_query_constructor_prompt(
    document_content_description,
    metadata_field_info,
)

# 구성 요소에서 구조화된 쿼리 출력 파서를 생성합니다.
output_parser = StructuredQueryOutputParser.from_components()

# 프롬프트, 언어 모델, 출력 파서를 연결하여 쿼리 생성기를 만듭니다.
query_constructor = prompt | llm | output_parser

`prompt.format()` 메서드를 사용하여 `query` 매개변수에 "dummy question" 문자열을 전달하고 그 결과를 출력하여 Prompt 내용을 확인해 보겠습니다.


In [33]:
# prompt 문자열에 query 매개변수를 "dummy question"으로 설정하여 포맷팅한 결과를 출력합니다.
print(prompt.format(query="dummy question"))

Your goal is to structure the user's query to match the request schema provided below.

<< Structured Request Schema >>
When responding use a markdown code snippet with a JSON object formatted in the following schema:

```json
{
    "query": string \ text string to compare to document contents
    "filter": string \ logical condition statement for filtering documents
}
```

The query string should contain only text that is expected to match the contents of documents. Any conditions in the filter should not be mentioned in the query as well.

A logical condition statement is composed of one or more comparison and logical operation statements.

A comparison statement takes the form: `comp(attr, val)`:
- `comp` (eq | ne | gt | gte | lt | lte | contain | like | in | nin): comparator
- `attr` (string):  name of attribute to apply the comparison to
- `val` (string): is the comparison value

A logical operation statement takes the form `op(statement1, statement2, ...)`:
- `op` (and | or | not

`query_constructor.invoke()` 메서드를 호출하여 주어진 쿼리에 대한 처리를 수행합니다.


In [34]:
query_constructor.invoke(
    {
        # 쿼리 생성기를 호출하여 주어진 질문에 대한 쿼리를 생성합니다.
        "query": "What are some sci-fi movies from the 90's directed by Luc Besson about taxi drivers"
    }
)

StructuredQuery(query='taxi drivers', filter=Operation(operator=<Operator.AND: 'and'>, arguments=[Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='genre', value='science fiction'), Comparison(comparator=<Comparator.GTE: 'gte'>, attribute='year', value=1990), Comparison(comparator=<Comparator.LTE: 'lte'>, attribute='year', value=1999), Comparison(comparator=<Comparator.EQ: 'eq'>, attribute='director', value='Luc Besson')]), limit=None)

Self-query retriever의 핵심 요소는 query constructor입니다. 훌륭한 검색 시스템을 만들기 위해서는 query constructor가 잘 작동하도록 해야 합니다.

이를 위해서는 **프롬프트(Prompt), 프롬프트 내의 예시, 속성 설명 등을 조정** 해야 합니다.

[참고]

- 호텔 인벤토리 데이터에 대한 query constructor를 개선하는 과정을 보여주는 예시는 [쿡북 튜토리얼](https://github.com/langchain-ai/langchain/blob/master/cookbook/self_query_hotel_search.ipynb)을 참조하세요.


### ② 구조화된 쿼리 변환기(Structured Query Translator)를 사용하여 구조화된 쿼리로 변환


다음으로 중요한 요소는 structured query translator입니다. 이는 일반적인 `StructuredQuery` 객체를 사용 중인 vector store의 구문에 맞는 메타데이터 필터로 변환하는 역할을 담당합니다.


`SelfQueryRetriever`를 사용하여 질문에 대한 답변을 생성하는 검색기를 구현합니다.

- `query_constructor`를 사용하여 질문을 구성합니다.
- `vectorstore`를 사용하여 벡터 저장소에 접근합니다.
- `ChromaTranslator`를 사용하여 구조화된 질의를 Chroma 벡터 저장소에 맞게 변환합니다.


In [11]:
from langchain.retrievers.self_query.chroma import ChromaTranslator

retriever = SelfQueryRetriever(
    query_constructor=query_constructor, # 이전에 생성한 쿼리 생성기
    vectorstore=vectorstore, # 벡터 저장소를 지정
    structured_query_translator=ChromaTranslator(), # 쿼리 변환기

`retriever.invoke()` 메서드를 사용하여 주어진 질문에 대한 답변을 생성합니다.


In [36]:
retriever.invoke(
    # 1990년 이후 2005년 이전에 제작된 장난감에 관한 영화를 검색하며, 애니메이션 영화가 선호됩니다.
    "What's a movie after 1990 but before 2005 that's all about toys, and preferably is animated"
)

[Document(page_content='Toys come alive and have a blast doing so', metadata={'genre': 'animated', 'year': 1995})]

이렇게 구조화된 쿼리 생성기 + 쿼리 변환기를 사용하여 오류 없이 데이터를 필터링하고 검색할 수 있습니다.
